In [5]:
import requests
import math

# An api key is emailed to you when you sign up to a plan
# Get a free API key at https://api.the-odds-api.com/
API_KEY = 'c146d41ee6fa4c4af149f4965849f905'

# americanfootball_nfl
# americanfootball_ncaaf
# baseball_mlb
SPORT = 'americanfootball_nfl' # use the sport_key from the /sports endpoint below, or use 'upcoming' to see the next 8 games across all sports

REGIONS = 'us,us2' # uk | us | eu | au. Multiple can be specified if comma delimited

# h2h
MARKETS = 'h2h' # h2h | spreads | totals. Multiple can be specified if comma delimited

ODDS_FORMAT = 'decimal' # decimal | american

DATE_FORMAT = 'iso' # iso | unix


In [6]:
ABBR_TO_NAME = {
    "ARI": "Arizona Cardinals",
    "ATL": "Atlanta Falcons",
    "BAL": "Baltimore Ravens",
    "BUF": "Buffalo Bills",
    "CAR": "Carolina Panthers",
    "CHI": "Chicago Bears",
    "CIN": "Cincinnati Bengals",
    "CLE": "Cleveland Browns",
    "DAL": "Dallas Cowboys",
    "DEN": "Denver Broncos",
    "DET": "Detroit Lions",
    "GB":  "Green Bay Packers",
    "HOU": "Houston Texans",
    "IND": "Indianapolis Colts",
    "JAX": "Jacksonville Jaguars",
    "KC":  "Kansas City Chiefs",
    "LV":  "Las Vegas Raiders",
    "LAC": "Los Angeles Chargers",
    "LAR": "Los Angeles Rams",
    "MIA": "Miami Dolphins",
    "MIN": "Minnesota Vikings",
    "NE":  "New England Patriots",
    "NO":  "New Orleans Saints",
    "NYG": "New York Giants",
    "NYJ": "New York Jets",
    "PHI": "Philadelphia Eagles",
    "PIT": "Pittsburgh Steelers",
    "SF":  "San Francisco 49ers",
    "SEA": "Seattle Seahawks",
    "TB":  "Tampa Bay Buccaneers",
    "TEN": "Tennessee Titans",
    "WAS": "Washington Commanders",
}


In [ ]:
def decimal_to_implied_prob(decimal_odds):
    return 1 / decimal_odds

def process_game(game, valid_bookmakers=[], print_output=False):
    if print_output:
        print(game['home_team'], 'vs', game['away_team'])
        print('Commence time:', game['commence_time'])
    team_a_name = game['home_team']
    team_b_name = game['away_team']
    lowest_team_a = 1.0
    lowest_team_a_bookermaker = 'n/a'
    lowest_team_b = 1.0
    lowest_team_b_bookermaker = 'n/a'
    for bookmaker in game['bookmakers']:
        if len(valid_bookmakers) != 0 and bookmaker['title'] not in valid_bookmakers:
            continue
        
        if print_output:
            print('Bookmaker:', bookmaker['title'])

        for market in bookmaker['markets']:
            if market['key'] != 'h2h':
                continue
            if print_output:
                print(' Market:', market['key'])
            if market['outcomes'][0]['name'] == team_a_name and market['outcomes'][1]['name'] == team_b_name:
                team_a = market['outcomes'][0]
                team_b = market['outcomes'][1]
            elif market['outcomes'][0]['name'] == team_b_name and market['outcomes'][1]['name'] == team_a_name:
                team_a = market['outcomes'][1]
                team_b = market['outcomes'][0]
            else:
                if print_output:
                    print('  Skipping market {team_a_name} vs {team_b_name}, team names do not match')
                continue
            team_a_implied_prob = decimal_to_implied_prob(team_a['price'])
            team_b_implied_prob = decimal_to_implied_prob(team_b['price'])
            if print_output:
                print('  Outcome:', team_a['name'], 'Price:', team_a_implied_prob)
                print('  Outcome:', team_b['name'], 'Price:', team_b_implied_prob)
            if team_a_implied_prob < lowest_team_a:
                lowest_team_a = team_a_implied_prob
                lowest_team_a_bookermaker = bookmaker['title']
            elif team_a_implied_prob == lowest_team_a:
                lowest_team_a_bookermaker += ', ' + bookmaker['title']
            if team_b_implied_prob < lowest_team_b:
                lowest_team_b = team_b_implied_prob
                lowest_team_b_bookermaker = bookmaker['title']
            elif team_b_implied_prob == lowest_team_b:
                lowest_team_b_bookermaker += ', ' + bookmaker['title']

    if print_output:
        print('Best odds across all bookmakers:')
        print(lowest_team_a_bookermaker, ' - ', game['home_team'], ' - implied probability:', lowest_team_a)
        print(lowest_team_b_bookermaker, ' - ', game['away_team'], ' - implied probability:', lowest_team_b)
    
    return {
        'home_team': game['home_team'],
        'away_team': game['away_team'],
        'commence_time': game['commence_time'],
        'best_odds': {
            game['home_team']: {
                'implied_prob': lowest_team_a,
                'bookmaker': lowest_team_a_bookermaker
            },
            game['away_team']: {
                'implied_prob': lowest_team_b,
                'bookmaker': lowest_team_b_bookermaker
            }
        }
    }

def round_up_nearest_cent(number):
  """
  Rounds a number up to the nearest hundredth.
  """
  return math.ceil(number * 100) / 100

def get_orderbook_url(market_ticker):
    return f"https://api.elections.kalshi.com/trade-api/v2/markets/{market_ticker}/orderbook"

def get_prices(orderbook_url):
  orderbook_response = requests.get(orderbook_url)
  orderbook_data = orderbook_response.json()

  highest_bid = orderbook_data['orderbook']['yes'][-1][0]
  highest_bid_volume = orderbook_data['orderbook']['yes'][-1][1]

  lowest_ask = 100 - orderbook_data['orderbook']['no'][-1][0]
  lowest_ask_volume = orderbook_data['orderbook']['no'][-1][1]

  return highest_bid, highest_bid_volume, lowest_ask, lowest_ask_volume

def get_prices_from_market(market):
    market_ticker = market['ticker']
    orderbook_url = get_orderbook_url(market_ticker)
    yes_bid, yes_bid_volume, yes_ask, yes_ask_volume = get_prices(orderbook_url)
    print(f"- {market['ticker'].split('-')[-1]}: Yes Bid: {yes_bid}¢ | Volume: {yes_bid_volume}, Yes Ask: {yes_ask}¢ | Volume: {yes_ask_volume}")
    return yes_bid, yes_bid_volume, yes_ask, yes_ask_volume

# Calculates the fair price to buy/sell a given number of contracts at the lowest ask/highest bid, including fees.
# Returns (taker_price, maker_price, taker_probability, maker_probability)
def fair_price(number_of_contracts, lowest_ask):
    lowest_ask_percentage = lowest_ask / 100
    highest_bid = lowest_ask - 1
    highest_bid_percentage = highest_bid / 100
    taker_odds = 0.07 * lowest_ask_percentage * (1 - lowest_ask_percentage)
    maker_odds = 0.0175 * highest_bid_percentage * (1 - highest_bid_percentage)
    taker_fees = round_up_nearest_cent(taker_odds * number_of_contracts)
    maker_fees = round_up_nearest_cent(maker_odds * number_of_contracts)

    return round(taker_fees + number_of_contracts * lowest_ask_percentage, 2),  round(maker_fees + number_of_contracts * highest_bid_percentage, 2), round(taker_odds + lowest_ask_percentage, 6), round(maker_odds + highest_bid_percentage, 6)

In [8]:
# Sports Books

odds_response = requests.get(
    f'https://api.the-odds-api.com/v4/sports/{SPORT}/odds',
    params={
        'api_key': API_KEY,
        'regions': REGIONS,
        'markets': MARKETS,
        'oddsFormat': ODDS_FORMAT,
        'dateFormat': DATE_FORMAT,
    }
)

if odds_response.status_code != 200:
    print(f'Failed to get odds: status_code {odds_response.status_code}, response body {odds_response.text}')

else:
    odds_json = odds_response.json()
    print('Number of events:', len(odds_json))
    print(odds_json)

    # Check the usage quota
    print('Remaining requests', odds_response.headers['x-requests-remaining'])
    print('Used requests', odds_response.headers['x-requests-used'])

Number of events: 32
[{'id': 'aca5234c57e31b1931e51d2d0d6046f5', 'sport_key': 'americanfootball_nfl', 'sport_title': 'NFL', 'commence_time': '2025-09-19T00:15:00Z', 'home_team': 'Buffalo Bills', 'away_team': 'Miami Dolphins', 'bookmakers': [{'key': 'draftkings', 'title': 'DraftKings', 'last_update': '2025-09-18T21:12:58Z', 'markets': [{'key': 'h2h', 'last_update': '2025-09-18T21:12:58Z', 'outcomes': [{'name': 'Buffalo Bills', 'price': 1.12}, {'name': 'Miami Dolphins', 'price': 6.75}]}]}, {'key': 'hardrockbet', 'title': 'Hard Rock Bet', 'last_update': '2025-09-18T21:13:39Z', 'markets': [{'key': 'h2h', 'last_update': '2025-09-18T21:13:39Z', 'outcomes': [{'name': 'Buffalo Bills', 'price': 1.11}, {'name': 'Miami Dolphins', 'price': 7.0}]}]}, {'key': 'espnbet', 'title': 'ESPN BET', 'last_update': '2025-09-18T21:12:59Z', 'markets': [{'key': 'h2h', 'last_update': '2025-09-18T21:12:59Z', 'outcomes': [{'name': 'Buffalo Bills', 'price': 1.12}, {'name': 'Miami Dolphins', 'price': 6.0}]}]}, {'key'

In [12]:
processed_game = process_game(odds_response.json()[1])
processed_game

{'home_team': 'Carolina Panthers',
 'away_team': 'Atlanta Falcons',
 'commence_time': '2025-09-21T17:00:00Z',
 'best_odds': {'Carolina Panthers': {'implied_prob': 0.684931506849315,
   'bookmaker': 'DraftKings'},
  'Atlanta Falcons': {'implied_prob': 0.31746031746031744,
   'bookmaker': 'Fliff'}}}

In [ ]:
valid_bookmakers = ['DraftKings', 'BetMGM', 'Caesars', 'FanDuel', 'ESPN BET']

for game in odds_response.json():
    processed_game = process_game(game, valid_bookmakers, print_output=False)
    



    sum = processed_game['best_odds'][processed_game['home_team']]['implied_prob'] + processed_game['best_odds'][processed_game['away_team']]['implied_prob']
    if sum < 2.015:
        print(processed_game['home_team'] + ' vs ' + processed_game['away_team'])
        print(processed_game['commence_time'])
        print('---')
        for team, odds in processed_game['best_odds'].items():
            print(odds['bookmaker'])
            print(team + ': ' + str(odds['implied_prob']))
        print(sum)
        print('######################################################################')